## Yahoo Fundamental

In [ ]:
# def get_yahoo_economic_calendar(table_class: str = None) -> pd.DataFrame:
#     """
#     Extrat data from trading economics callendar.

#     Parameters:
#         classe_tabela (str): Class CSS from table to scrap (optional).

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get("https://tradingeconomics.com/calendar", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')

#         if table_class:
#             tabela = soup.find('table', {'class': table_class})
#         else:
#             tabela = soup.find('table')
        
#         headers = [th.text.strip() for th in tabela.find_all('th')]
#         print(headers)
#         rows = []
#         for row in tabela.find_all('tr')[1:]:
#             cols = [td.text.strip() for td in row.find_all('td')]
#             if cols: 
#                 rows.append(cols)

#         df = pd.DataFrame(rows, columns=headers if headers else None)
        

#     except requests.exceptions.RequestException as e:
#         print(f"Error accessing URL: {e}")
#     except Exception as e:
#         print(f"Error processing data: {e}")

#     return df


In [ ]:
# def get_whalewisdom(ticker: str) -> pd.DataFrame:
#     """
#     Extracts data from WhaleWisdom for a given stock ticker.

#     Parameters:
#         ticker (str): Stock ticker symbol.

#     Returns:
#         pd.DataFrame: DataFrame with table content.
#     """

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     try:
#         response = requests.get(f"https://whalewisdom.com/stock/{ticker}", headers=headers)
#         response.raise_for_status()

#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         # Encontrar a div com a classe "v-data-table__wrapper"
#         div_wrapper = soup.find('div', {'class': 'v-window__container'})
       
#         if not div_wrapper:
#             raise ValueError("Div com a classe 'v-window__container' não encontrada.")

#         # Buscar a primeira tabela dentro desta div
#         tabela = div_wrapper.find('table')
#         if not tabela:
#             raise ValueError("Nenhuma tabela encontrada dentro da div especificada.")
        
#         # Coletar os cabeçalhos da tabela ignorando ícones
#         headers = [th.find('span').text.strip() for th in tabela.find_all('th') if th.find('span')]
#         # print(headers)

#         # Encontrando todas as linhas dentro da tabela (tbody > tr)
#         rows = tabela.find("tbody").find_all("tr")
#         print(rows)
#         # Extraindo os dados de cada linha
#         data = []
#         for row in rows:
#             columns = row.find_all("td")

#             # Pegando os valores correspondentes
#             institution_name = columns[0].text.strip()
#             shares_held = columns[2].text.strip()
#             portfolio_value = columns[3].text.strip()
#             percentage_ownership = columns[4].text.strip()
#             last_report_date = columns[-1].text.strip()

#             data.append({
#                 "Instituição": institution_name,
#                 "Ações Detidas": shares_held,
#                 "Valor do Portfólio": portfolio_value,
#                 "Percentual de Participação": percentage_ownership,
#                 "Última Atualização": last_report_date
#             })
#         #print(data)
    
#     except requests.exceptions.RequestException as e:
#         print(f"Erro ao acessar a URL: {e}")
#     except ValueError as e:
#         print(f"Erro na extração de dados: {e}")
#     except Exception as e:
#         print(f"Erro inesperado: {e}")
    
#     return pd.DataFrame()

In [ ]:
# df_whale_s = get_whalewisdom(ticker = "ctre")
# df_whale_s

NameError: name 'requests' is not defined

In [ ]:
# base_url = 'https://whalewisdom.com/shell/command.html?args=%7B%22command%22:%22filer_lookup%22,%20%22name%22:%22berkshire%22%7D'
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
#     "Accept": "application/json"
# }

# response = requests.get(base_url, headers=headers)
# response.raise_for_status()

# response.text

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <title>Track Hedge Funds Using 13F Filings </title>\n    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, minimal-ui">\n            <meta name="description" content="WhaleWisdom tracks 13F,  Schedule 13D, and 13G EDGAR filings by hedge funds. Hedge Fund Whale Backtesting and search tools "/>\n    <link rel="shortcut icon" type="image/x-icon" href="https://d27mjrcvcy56qq.cloudfront.net/images/favicon.ico"/>\n    <link rel="apple-touch-icon" sizes="57x57" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-57x57.png">\n    <link rel="apple-touch-icon" sizes="60x60" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-60x60.png">\n    <link rel="apple-touch-icon" sizes="72x72" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-72x72.png">\n    <link rel="apple-touch-icon" sizes="76x76" href="https://d27mjrcvcy56

In [ ]:
# import hashlib
# import hmac
# import base64
# import json
# import time
# import urllib.parse
# import requests

# class WhaleWisdom:
#     def __init__(self, secret_key, shared_key):
#         self.secret_key = secret_key
#         self.shared_key = shared_key
#         self.timestamp = self.get_timestamp()

#     def get_timestamp(self):
#         """Gera o timestamp no formato UTC"""
#         return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

#     def signature(self, args):
#         """Gera a assinatura HMAC-SHA1 em Base64"""
#         message = f"{args}\n{self.timestamp}".encode('utf-8')
#         hmac_digest = hmac.new(self.secret_key.encode('utf-8'), message, hashlib.sha1).digest()
#         return base64.b64encode(hmac_digest).decode('utf-8')

#     def encode(self, string):
#         """Codifica a string para URL"""
#         return urllib.parse.quote(string, safe='')

#     def endpoint(self, args):
#         """Constrói a URL da API do WhaleWisdom"""
#         encoded_args = self.encode(json.dumps(args))
#         api_sig = self.signature(json.dumps(args))
#         encoded_timestamp = self.encode(self.timestamp)

#         return f"https://whalewisdom.com/shell/command.json?args={encoded_args}&api_shared_key={self.shared_key}&api_sig={api_sig}&timestamp={encoded_timestamp}"

#     def request(self, args):
#         """Faz a requisição à API"""
#         url = self.endpoint(args)
#         response = requests.get(url)

#         if response.status_code == 200:
#             return response.text
#         else:
#             print(f"Erro {response.status_code}: {response.text}")
#             return None


# # **Exemplo de Uso**
# WW_SHARED_KEY = "aTWxWrcILEOGqkAPZtAL"
# WW_SECRET_KEY = "JGlAleUrGy1bWaLoiWwESNIbTHaovA0ocKOnr4MW"

# ww = WhaleWisdom(secret_key=WW_SECRET_KEY, shared_key=WW_SHARED_KEY)

# # Obter as participações (holdings) de um gestor específico
# response = ww.request({
#     "command": "stock_lookup",
#     "symbol": "ctre"
# })

# data = json.loads(response)

# stock_id = data["stocks"][0]["id"]
# name = data["stocks"][0]["name"]
# status = data["stocks"][0]["status"]
# link = data["stocks"][0]["link"]

# stock_id

167941

In [ ]:
# response = ww.request(
# {"command":"holdings","stock_ids":[167941],"filer_ids":[373], "limit": 10, "all_quarters":0})
# response


'{"errors":["Subscription limit has been reached for current period"]}'

In [ ]:
# response

'{"errors":["Subscription limit has been reached for current period"]}'

## STOCKS



In [1]:
import yfinance as yf
import pandas as pd
import talib
from django.http import JsonResponse
from datetime import datetime, timedelta

# S&P 500 → ^GSPC
# Dow Jones → ^DJI
# Nasdaq 100 → ^NDX
# Russell 2000 → ^RUT
# DAX (Alemanha) → ^GDAXI
# Definir o ticker da ação
symbol = "AAPL"  # Apple Inc.
# symbol = "^GSPC"  # Apple Inc.
# symbol = "XLK"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários

yahoo_symbol_info = yf.Ticker(symbol).info

In [2]:
def get_sector_etf_info(sector_name, search_value: str = "info" ):
    """
    Return information about symbol ETF sector.
    """
    sector_map = {
        "Technology": "XLK",
        "Financial Services": "XLF",
        "Consumer Cyclical": "XLY",
        "Healthcare": "XLV",
        "Communication Services": "XLC",
        "Industrials": "XLI",
        "Consumer Defensive": "XLP",
        "Energy": "XLE",
        "Real Estate": "XLRE",
        "Basic Materials": "XLB",
        "Utilities": "XLU",
    }

    etf_symbol = sector_map.get(sector_name)
    if not etf_symbol:
        return "N/A"

    ticker = yf.Ticker(etf_symbol)

    if search_value == "info":
        return ticker.info
    else:
        return ticker.info.get(search_value, "N/A")


In [4]:
try:
    yahoo_symbol_income = yf.Ticker(symbol).income_stmt
except:
    yahoo_symbol_income = pd.DataFrame()

# Inicializar todas as variáveis com "N/A"
net_income = "N/A"
total_revenue = "N/A"
cost_of_revenue = "N/A"
gross_profit = "N/A"

# net_income
if 'Net Income' in yahoo_symbol_income.index:
    net_income = yahoo_symbol_income.loc['Net Income']
    if pd.isna(net_income).all():
        net_income = "N/A"

# total_revenue
if 'Total Revenue' in yahoo_symbol_income.index:
    total_revenue = yahoo_symbol_income.loc['Total Revenue']
    if pd.isna(total_revenue).all():
        total_revenue = "N/A"

# cost_of_revenue
if 'Cost Of Revenue' in yahoo_symbol_income.index:
    cost_of_revenue = yahoo_symbol_income.loc['Cost Of Revenue']
    if pd.isna(cost_of_revenue).all():
        cost_of_revenue = "N/A"

# gross_profit
if 'Gross Profit' in yahoo_symbol_income.index:
    gross_profit = yahoo_symbol_income.loc['Gross Profit']
    if pd.isna(gross_profit).all():
        gross_profit = "N/A"

In [5]:
def get_yoy_metric(series):
    if isinstance(series, pd.Series):
        yoy_growth = ((series - series.shift(-1)) / series.shift(-1)) * 100
        yoy_growth = yoy_growth.dropna()
        
    return yoy_growth

def get_cagr_metric(series):
    series = series.dropna()
    if len(series) >= 2:
        start_value = series.iloc[-1]
        end_value = series.iloc[0]
        num_years = len(series) - 1
        cagr = ((end_value / start_value) ** (1 / num_years)) - 1

    cagr_percent = cagr * 100 if cagr != "N/A" else "N/A"

    return cagr_percent

In [18]:
yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
yahoo_symbol_cashflow.index

Index(['Free Cash Flow', 'Repurchase Of Capital Stock', 'Repayment Of Debt',
       'Issuance Of Debt', 'Issuance Of Capital Stock', 'Capital Expenditure',
       'Interest Paid Supplemental Data', 'Income Tax Paid Supplemental Data',
       'End Cash Position', 'Beginning Cash Position', 'Changes In Cash',
       'Financing Cash Flow', 'Cash Flow From Continuing Financing Activities',
       'Net Other Financing Charges', 'Cash Dividends Paid',
       'Common Stock Dividend Paid', 'Net Common Stock Issuance',
       'Common Stock Payments', 'Common Stock Issuance',
       'Net Issuance Payments Of Debt', 'Net Short Term Debt Issuance',
       'Net Long Term Debt Issuance', 'Long Term Debt Payments',
       'Long Term Debt Issuance', 'Investing Cash Flow',
       'Cash Flow From Continuing Investing Activities',
       'Net Other Investing Changes', 'Net Investment Purchase And Sale',
       'Sale Of Investment', 'Purchase Of Investment',
       'Net Business Purchase And Sale', 'Purch

In [26]:
free_cashflow = yahoo_symbol_cashflow.loc['Free Cash Flow']
get_cagr_metric(free_cashflow)

operating_cashflow = yahoo_symbol_cashflow.loc['Operating Cash Flow']
capital_expenditure = yahoo_symbol_cashflow.loc['Capital Expenditure']

In [28]:
free_cashflow.iloc[0]
operating_cashflow.iloc[0]
capital_expenditure.iloc[0]

result = operating_cashflow.iloc[0]-abs(capital_expenditure.iloc[0]) 

free_cashflow.iloc[0], operating_cashflow.iloc[0], capital_expenditure.iloc[0], result

(108807000000.0, 118254000000.0, -9447000000.0, 108807000000.0)

In [9]:
# yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
# yahoo_symbol_cashflow.index

In [10]:
# revenue_history = yahoo_symbol_info.get("netIncomeToCommon")
# revenue_history

In [11]:
# https://medium.com/towards-data-science/pivot-points-calculation-in-python-for-day-trading-659c1e92d323


# ### ZONAS PIVOT! ###

## Data Collection Binance


In [ ]:
# import requests

# url = "https://api.binance.com/api/v3/ticker/price"
# params = {"symbol": "BTCUSDT"}

# response = requests.get(url, params=params)

# if response.status_code == 200:
#     symbol = response.json()["symbol"]
#     price = response.json()["price"]
# else:
#     error = response.status_code

In [ ]:
# from decimal import Decimal
# from datetime import datetime


# def get_crypto_symbol_24h(symbol : str):

#     url = "https://api.binance.com/api/v3/ticker/24hr"
#     params = {"symbol": symbol}

#     response = requests.get(url, params=params)

#     if response.status_code == 200:
#         priceChangePercent = Decimal(response.json()["priceChangePercent"])
#         weightedAvgPrice = Decimal(response.json()["weightedAvgPrice"])
#         prevClosePrice = Decimal(response.json()["prevClosePrice"])
#         priceChange = Decimal(response.json()["priceChange"])
#         lastPrice = Decimal(response.json()["lastPrice"])
#         lastQty = Decimal(response.json()["lastQty"])
#         bidPrice = Decimal(response.json()["bidPrice"])
#         bidQty = Decimal(response.json()["bidQty"])
#         askPrice = Decimal(response.json()["askPrice"])
#         askQty = Decimal(response.json()["askQty"])
#         openPrice = Decimal(response.json()["openPrice"])
#         highPrice = Decimal(response.json()["highPrice"])
#         lowPrice = Decimal(response.json()["lowPrice"])
#         volume = Decimal(response.json()["volume"])
#         quoteVolume = Decimal(response.json()["quoteVolume"])
#         openTime = datetime.fromtimestamp(response.json()["openTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
#         closeTime = datetime.fromtimestamp(response.json()["closeTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
#         firstId = response.json()["firstId"]
#         lastId = response.json()["lastId"]
#         count = response.json()["count"]
#     else:
#         print(f"Erro: {response.status_code} - {response.text}")

In [ ]:
# url = "https://api.binance.com/api/v3/openOrders"
# params = {"symbol": "BTCUSDT"}

# response = requests.get(url, params=params)
# response.json()
